Revoir minimum WR distances : très long 

In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from swiss_rounds_simulator.utils import simulate_tournament, simulate_n_tournaments, compare_settings
from swiss_rounds_simulator.utils import  initiate_league, assign_opponents, play_round
from swiss_rounds_simulator.mwmatching import maxWeightMatching


New LDC format places of interest :
* The 8 first teams qualify directly to round of 16
* From 9 to 24 : pseudo round of 32
* Interesting teams : i (9th) and y(25th)  

Format :
* 36 teams,
* 8 games,
* Hat determination of the first 2 games

LoL :
* 16 teams,
* Top 8 to Quarter finales,
* 6 games (not exactly, teams with 3 win qualifies, team with 3 loss gets eliminated)


In [31]:
def rank_table(league_table:pd.DataFrame, n_rounds:int):
    if 'rank' not in league_table.columns:
        league_table['rank'] = [0] * league_table.shape[0]

    init_ranks = league_table['rank']
    keep_ranking=True
    use_OWR = False
    j = 1
    
    while keep_ranking:
        multi_given_ranks = init_ranks.value_counts()[init_ranks.value_counts()>1].index
        if not multi_given_ranks.shape[0] > 0:
            keep_ranking = False        
        n_unranked_init = init_ranks.value_counts()[init_ranks.value_counts()>1].sum()
        print(f"Iteration number {j}, {use_OWR=}, number of unranked teams = {n_unranked_init} ")
        for initial_rank in multi_given_ranks :
            subcomp_df = league_table.query(f"rank =={initial_rank}")
            subcomps = []
            for score in subcomp_df['Nb_win'].unique():
                subtable = subcomp_df.query(f"Nb_win == {score}")
                for team in subtable.index:
                    team_subcomp_games = []
                    for i in range(n_rounds):
                        opponent = subcomp_df.loc[team,f'R{i+1}_opponent']
                        if opponent in subtable.index:
                            result = subcomp_df.loc[team,f'R{i+1}_result']
                            team_subcomp_games.append(1 if result == 'Win' else 0)
                    team_subscore = np.mean(team_subcomp_games) if len(team_subcomp_games) >0 else 0.5
                    subcomps.append({'team':team, 'subscore':team_subscore})
            tmp_df = pd.DataFrame(subcomps).set_index('team')
            subcomp_df = subcomp_df.merge(tmp_df, left_index=True, right_index=True)
            subcomp_df['tmp_rank'] = 1000000 * subcomp_df['Nb_win'] + 1000 * subcomp_df['subscore'] + (subcomp_df['OWR'] if use_OWR else 0)
            display(subcomp_df)
            r_df = subcomp_df['tmp_rank'].sort_values(ascending=False).drop_duplicates().reset_index().drop(columns = 'index').reset_index().set_index('tmp_rank')
            
            for team in subcomp_df.index:
                league_table.loc[team,'rank'] = r_df.loc[subcomp_df.loc[team,'tmp_rank'],'index'] + initial_rank
        
        final_ranks = league_table['rank']
        n_unranked_final = final_ranks.value_counts()[final_ranks.value_counts()>1].sum()
        if n_unranked_init == n_unranked_final:
            print("Test on ranks = True")
            if use_OWR :
                keep_ranking = False
            else :
                use_OWR = True
                init_ranks = final_ranks    
                j+=1
        else :
            print("Test on ranks = False")
            init_ranks = final_ranks    
            j+=1

    league_table.sort_values(by='rank', inplace = True)
    league_table['rank'] = league_table['rank'] +1
    #league_table[['Win_rate','OWR']] = league_table[['Win_rate','OWR']].apply(lambda x: np.round(x,2))
    return league_table


In [32]:
n_teams = 16
n_rounds = 6

lt = simulate_tournament(nb_teams=n_teams, nb_games=n_rounds, method='probabilistic', delta_level='linear', verbose = False)
lt = rank_table(lt,n_rounds)
for i in range(n_rounds+1):
    display(lt.query(f"Nb_win == {n_rounds-i}"))

Iteration number 1, use_OWR=False, number of unranked teams = 16 


,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opponent,R1_result,R2_opponent,...,R3_result,R4_opponent,R4_result,R5_opponent,R5_result,R6_opponent,R6_result,rank,subscore,tmp_rank
e,4,0.75,[],5,1,0.833333,0.583333,j,Win,c,...,Win,b,Win,f,Win,d,Loss,0,0.500000,5000500.0
d,3,0.8125,[],4,2,0.666667,0.611111,k,Win,b,...,Win,g,Win,h,Loss,e,Win,0,0.333333,4000333.333333
b,1,0.9375,[],4,2,0.666667,0.583333,p,Win,d,...,Win,e,Loss,c,Win,l,Loss,0,0.666667,4000666.666667
c,2,0.875,[],4,2,0.666667,0.555556,o,Win,e,...,Win,a,Win,b,Loss,k,Win,0,0.666667,4000666.666667
h,7,0.5625,[],4,2,0.666667,0.5,a,Loss,p,...,Loss,i,Win,d,Win,f,Win,0,0.333333,4000333.333333
a,0,1.0,[],4,2,0.666667,0.5,h,Win,f,...,Win,c,Loss,k,Win,g,Win,0,0.500000,4000500.0
l,11,0.3125,[],4,2,0.666667,0.472222,f,Loss,i,...,Loss,o,Win,j,Win,b,Win,0,0.500000,4000500.0
f,5,0.6875,[],3,3,0.5,0.666667,l,Win,a,...,Loss,j,Win,e,Loss,h,Loss,0,1.000000,3001000.0
g,6,0.625,[],3,3,0.5,0.5,i,Win,m,...,Loss,d,Loss,o,Win,a,Loss,0,0.500000,3000500.0
j,9,0.4375,[],3,3,0.5,0.472222,e,Loss,o,...,Win,f,Loss,l,Loss,n,Win,0,0.000000,3000000.0


Test on ranks = False
Iteration number 2, use_OWR=False, number of unranked teams = 8 


,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opponent,R1_result,R2_opponent,...,R3_result,R4_opponent,R4_result,R5_opponent,R5_result,R6_opponent,R6_result,rank,subscore,tmp_rank
d,3,0.8125,[],4,2,0.666667,0.611111,k,Win,b,...,Win,g,Win,h,Loss,e,Win,3,0.0,4000000.0
h,7,0.5625,[],4,2,0.666667,0.5,a,Loss,p,...,Loss,i,Win,d,Win,f,Win,3,1.0,4001000.0


,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opponent,R1_result,R2_opponent,...,R3_result,R4_opponent,R4_result,R5_opponent,R5_result,R6_opponent,R6_result,rank,subscore,tmp_rank
b,1,0.9375,[],4,2,0.666667,0.583333,p,Win,d,...,Win,e,Loss,c,Win,l,Loss,1,1.0,4001000.0
c,2,0.875,[],4,2,0.666667,0.555556,o,Win,e,...,Win,a,Win,b,Loss,k,Win,1,0.0,4000000.0


...

,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opponent,R1_result,R2_opponent,R2_result,R3_opponent,R3_result,R4_opponent,R4_result,R5_opponent,R5_result,R6_opponent,R6_result,rank
b,1,0.9375,[],4,2,0.666667,0.583333,p,Win,d,Win,f,Win,e,Loss,c,Win,l,Loss,2
c,2,0.875,[],4,2,0.666667,0.555556,o,Win,e,Loss,h,Win,a,Win,b,Loss,k,Win,3
h,7,0.5625,[],4,2,0.666667,0.5,a,Loss,p,Win,c,Loss,i,Win,d,Win,f,Win,4
d,3,0.8125,[],4,2,0.666667,0.611111,k,Win,b,Loss,l,Win,g,Win,h,Loss,e,Win,5
l,11,0.3125,[],4,2,0.666667,0.472222,f,Loss,i,Win,d,Loss,o,Win,j,Win,b,Win,5
a,0,1.0,[],4,2,0.666667,0.5,h,Win,f,Loss,m,Win,c,Loss,k,Win,g,Win,6


,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opponent,R1_result,R2_opponent,R2_result,R3_opponent,R3_result,R4_opponent,R4_result,R5_opponent,R5_result,R6_opponent,R6_result,rank
f,5,0.6875,[],3,3,0.5,0.666667,l,Win,a,Win,b,Loss,j,Win,e,Loss,h,Loss,7
j,9,0.4375,[],3,3,0.5,0.472222,e,Loss,o,Win,k,Win,f,Loss,l,Loss,n,Win,7
g,6,0.625,[],3,3,0.5,0.5,i,Win,m,Win,e,Loss,d,Loss,o,Win,a,Loss,8


,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opponent,R1_result,R2_opponent,R2_result,R3_opponent,R3_result,R4_opponent,R4_result,R5_opponent,R5_result,R6_opponent,R6_result,rank
k,10,0.375,[],2,4,0.333333,0.527778,d,Loss,n,Win,j,Loss,m,Win,a,Loss,c,Loss,8
i,8,0.5,[],2,4,0.333333,0.444444,g,Loss,l,Loss,p,Win,h,Loss,n,Loss,m,Win,9
m,12,0.25,[],2,4,0.333333,0.388889,n,Win,g,Loss,a,Loss,k,Loss,p,Win,i,Loss,10
n,13,0.1875,[],2,4,0.333333,0.305556,m,Loss,k,Loss,o,Loss,p,Win,i,Win,j,Loss,11


,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opponent,R1_result,R2_opponent,R2_result,R3_opponent,R3_result,R4_opponent,R4_result,R5_opponent,R5_result,R6_opponent,R6_result,rank
p,15,0.0625,[],1,5,0.166667,0.416667,b,Loss,h,Loss,i,Loss,n,Loss,m,Loss,o,Win,12
o,14,0.125,[],1,5,0.166667,0.472222,c,Loss,j,Loss,n,Win,l,Loss,g,Loss,p,Loss,13


,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opponent,R1_result,R2_opponent,R2_result,R3_opponent,R3_result,R4_opponent,R4_result,R5_opponent,R5_result,R6_opponent,R6_result,rank


In [22]:
comp = False
if comp :
    compare_settings(n_tournaments=100, 
                 n_teams=36, 
                 n_rounds = 8, 
                 delta_level ='linear',
                 strategies={'i':[1]}, 
                 probabilistic=True, 
                 deterministic=True)
